<a href="https://colab.research.google.com/github/howard-haowen/NLP-demos/blob/main/NQU_model_aided_lang_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 前置作業

In [ ]:
!pip install -U -q pip setuptools wheel
!pip install -U -q spacy
!python -m spacy download zh_core_web_sm
!pip install -q dragonmapper

In [2]:
!git clone -l -s https://github.com/L706077/jieba-zh_TW.git jieba_tw
%cd jieba_tw
import jieba
%cd ../

Cloning into 'jieba_tw'...
remote: Enumerating objects: 2320, done.
remote: Total 2320 (delta 0), reused 0 (delta 0), pack-reused 2320
Receiving objects: 100% (2320/2320), 36.07 MiB | 19.02 MiB/s, done.
Resolving deltas: 100% (1498/1498), done.
/content/jieba_tw
/content


# 調用模型

In [3]:
!pip show spacy

Name: spacy
Version: 3.3.0
Summary: Industrial-strength Natural Language Processing (NLP) in Python
Home-page: https://spacy.io
Author: Explosion
Author-email: contact@explosion.ai
License: MIT
Location: /usr/local/lib/python3.7/dist-packages
Requires: blis, catalogue, cymem, jinja2, langcodes, murmurhash, numpy, packaging, pathy, preshed, pydantic, requests, setuptools, spacy-legacy, spacy-loggers, srsly, thinc, tqdm, typer, typing-extensions, wasabi
Required-by: en-core-web-sm, fastai, zh-core-web-sm


In [4]:
from collections import Counter
from dragonmapper import hanzi, transcriptions
import pandas as pd
from pprint import pprint
import requests
import spacy
from spacy import displacy
from spacy.tokens import Doc

In [5]:
pd.options.plotting.backend = "plotly"

In [6]:
nlp = spacy.load("zh_core_web_sm")

class TwTokenizer:
    def __init__(self, vocab):
        self.vocab = vocab

    def __call__(self, text):
        words =  jieba.lcut(text)
        spaces = [False] * len(words)        
        return Doc(self.vocab, words=words, spaces=spaces)

nlp.tokenizer = TwTokenizer(nlp.vocab)

In [7]:
nlp.pipe_names

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'ner']

## 輸入文字

In [8]:
text = "阿伯說他要從台北騎腳踏車到高雄。"
doc = nlp(text)

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.953 seconds.
Prefix dict has been built succesfully.


## 斷詞

In [9]:
for token in doc:
    print(token.text, end=" | ")

阿伯 | 說 | 他 | 要 | 從 | 台北 | 騎 | 腳踏車 | 到 | 高雄 | 。 | 

## 命名實體

In [10]:
text = "迎接虎年到來，台北101今天表示，即日起推出「虎年新春燈光秀」，將持續至2月5日，每晚6時至10時，除整點會有報時燈光變化外，每15分鐘還會有3分鐘的燈光秀。台北101下午透過新聞稿表示，今年特別設計「虎年新春燈光秀」，從今晚開始閃耀台北天際線，一直延續至2月5日，共7天。"
doc = nlp(text)
displacy.render(doc, style='ent',jupyter=True)

# 漢字轉發音

In [28]:
text = "阿伯說他要從台北騎腳踏車到高雄。"
doc = nlp(text)
tokens = [token.text for token in doc]
tokens

['阿伯', '說', '他', '要', '從', '台北', '騎', '腳踏車', '到', '高雄', '。']

In [29]:
pinyins = [hanzi.to_pinyin(word) for word in tokens]
for pinyin in pinyins:
    print(pinyin, end=" | ")

ābó | shuō | tā | yào | cóng | Táiběi | qí | jiǎotàchē | dào | Gāoxióng | 。 | 

In [30]:
zhuyins = [transcriptions.pinyin_to_zhuyin(word) for word in pinyins]
for zhuyin in zhuyins:
    print(zhuyin, end=" | ")

ㄚ ㄅㄛˊ | ㄕㄨㄛ | ㄊㄚ | ㄧㄠˋ | ㄘㄨㄥˊ | ㄊㄞˊ ㄅㄟˇ | ㄑㄧˊ | ㄐㄧㄠˇ ㄊㄚˋ ㄔㄜ | ㄉㄠˋ | ㄍㄠ ㄒㄩㄥˊ | 。 | 

In [31]:
ipas = [transcriptions.pinyin_to_ipa(word) for word in pinyins]
for ipa in ipas:
    print(ipa, end=" | ")

a˥ pwɔ˧˥ | ʂwɔ˥ | tʰa˥ | jɑʊ˥˩ | tsʰʊŋ˧˥ | tʰaɪ˧˥ peɪ˧˩˧ | tɕʰi˧˥ | tɕjɑʊ˧˩˧ tʰa˥˩ ʈʂʰɤ˥ | tɑʊ˥˩ | kɑʊ˥ ɕjʊŋ˧˥ | 。 | 

# 查詢萌典

萌典範例: [點我](https://www.moedict.tw/%E9%AB%98%E8%88%88)

In [32]:
def moedict_caller(word):
    req = requests.get(f"https://www.moedict.tw/uni/{word}.json")
    if req:
        pprint(req.json())
    else:
        print("查無結果")

In [33]:
moedict_caller('高興')

{'heteronyms': [{'bopomofo': 'ㄍㄠ ㄒㄧㄥˋ',
                 'bopomofo2': 'gāu shìng',
                 'definitions': [{'def': '興趣甚高。',
                                  'quote': ['文選．殷仲文．南州桓公九井作詩：「獨有清秋日，能使高興盡。」',
                                            '唐．杜甫．北征詩：「青雲動高興，幽事亦可悅。」']},
                                 {'antonyms': '敗興,悲傷,悲哀,煩悶,難過,苦惱,沮喪,傷心,掃興,憂愁,厭惡,鬱悶',
                                  'def': '歡喜、愉快。',
                                  'example': ['如：「今天玩得高興嗎？」'],
                                  'synonyms': '得意,痛快,開心,快樂,快活,歡樂,歡喜,歡躍,喜悅,興奮,怡悅,愉快'},
                                 {'antonyms': '不快,生氣',
                                  'def': '以興致作決定事情的根據。',
                                  'example': ['如：「他高興，就讓他先走吧！」'],
                                  'synonyms': '樂意,願意'}],
                 'pinyin': 'gāo xìng'}],
 'title': '高興'}


In [34]:
def moedict_caller(word):
    req = requests.get(f"https://www.moedict.tw/uni/{word}.json")
    if req:
        definitions = req.json().get('heteronyms')[0].get('definitions')
        pprint(definitions)
    else:
        print("查無結果")

In [35]:
moedict_caller('高興')

[{'def': '興趣甚高。',
  'quote': ['文選．殷仲文．南州桓公九井作詩：「獨有清秋日，能使高興盡。」', '唐．杜甫．北征詩：「青雲動高興，幽事亦可悅。」']},
 {'antonyms': '敗興,悲傷,悲哀,煩悶,難過,苦惱,沮喪,傷心,掃興,憂愁,厭惡,鬱悶',
  'def': '歡喜、愉快。',
  'example': ['如：「今天玩得高興嗎？」'],
  'synonyms': '得意,痛快,開心,快樂,快活,歡樂,歡喜,歡躍,喜悅,興奮,怡悅,愉快'},
 {'antonyms': '不快,生氣',
  'def': '以興致作決定事情的根據。',
  'example': ['如：「他高興，就讓他先走吧！」'],
  'synonyms': '樂意,願意'}]


In [36]:
def moedict_caller(word):
    req = requests.get(f"https://www.moedict.tw/uni/{word}.json")
    if req:
        definitions = req.json().get('heteronyms')[0].get('definitions')
        df = pd.DataFrame(definitions)
        df.fillna("---", inplace=True)
        if 'example' not in df.columns:
            df['example'] = '---'
        if 'synonyms' not in df.columns:
            df['synonyms'] = '---' 
        if 'antonyms' not in df.columns:
            df['antonyms'] = '---' 
        cols = ['def', 'example', 'synonyms', 'antonyms']
        df = df[cols]
        df.rename(columns={
            'def': '解釋',
            'example': '例句',
            'synonyms': '同義詞',
            'antonyms': '反義詞',
        }, inplace=True)
        return df
    else:
        print("查無結果")

In [37]:
moedict_caller('高興')

,解釋,例句,同義詞,反義詞
0,興趣甚高。,---,---,---
1,歡喜、愉快。,[如：「今天玩得高興嗎？」],"得意,痛快,開心,快樂,快活,歡樂,歡喜,歡躍,喜悅,興奮,怡悅,愉快","敗興,悲傷,悲哀,煩悶,難過,苦惱,沮喪,傷心,掃興,憂愁,厭惡,鬱悶"
2,以興致作決定事情的根據。,[如：「他高興，就讓他先走吧！」],"樂意,願意","不快,生氣"


# 詞頻統計

In [39]:
text = "迎接虎年到來，台北101今天表示，即日起推出「虎年新春燈光秀」，將持續至2月5日，每晚6時至10時，除整點會有報時燈光變化外，每15分鐘還會有3分鐘的燈光秀。台北101下午透過新聞稿表示，今年特別設計「虎年新春燈光秀」，從今晚開始閃耀台北天際線，一直延續至2月5日，共7天。"
doc = nlp(text)
for token in doc:
    print(token.text, end=" | ")

迎接 | 虎年 | 到來 | ， | 台北 | 101 | 今天 | 表示 | ， | 即日 | 起 | 推出 | 「 | 虎年 | 新春 | 燈光秀 | 」 | ， | 將 | 持續 | 至 | 2 | 月 | 5 | 日 | ， | 每晚 | 6 | 時 | 至 | 10 | 時 | ， | 除整 | 點會 | 有 | 報時 | 燈光 | 變化 | 外 | ， | 每 | 15 | 分鐘 | 還 | 會 | 有 | 3 | 分鐘 | 的 | 燈光秀 | 。 | 台北 | 101 | 下午 | 透過 | 新聞稿 | 表示 | ， | 今年 | 特別 | 設計 | 「 | 虎年 | 新春 | 燈光秀 | 」 | ， | 從 | 今晚 | 開始 | 閃耀 | 台北 | 天際線 | ， | 一直 | 延續 | 至 | 2 | 月 | 5 | 日 | ， | 共 | 7 | 天 | 。 | 

In [40]:
PUNCT_SYM = ["PUNCT", "SYM"]

def filter_tokens(doc):
    clean_tokens = [tok for tok in doc if tok.pos_ not in PUNCT_SYM]
    clean_tokens = (
        [tok for tok in clean_tokens if 
         not tok.like_email and 
         not tok.like_num and 
         not tok.like_url and 
         not tok.is_space]
    )
    return clean_tokens

In [41]:
clean_tokens = filter_tokens(doc)
for token in clean_tokens:
    print(token.text, end=" | ")

迎接 | 虎年 | 到來 | 台北 | 今天 | 表示 | 即日 | 起 | 推出 | 虎年 | 新春 | 燈光秀 | 將 | 持續 | 至 | 月 | 日 | 每晚 | 時 | 至 | 時 | 除整 | 點會 | 有 | 報時 | 燈光 | 變化 | 外 | 每 | 分鐘 | 還 | 會 | 有 | 分鐘 | 的 | 燈光秀 | 台北 | 下午 | 透過 | 新聞稿 | 表示 | 今年 | 特別 | 設計 | 虎年 | 新春 | 燈光秀 | 從 | 今晚 | 開始 | 閃耀 | 台北 | 天際線 | 一直 | 延續 | 至 | 月 | 日 | 共 | 天 | 

In [42]:
tokens = [token.text for token in clean_tokens]
counter = Counter(tokens)
counter

Counter({'一直': 1,
         '下午': 1,
         '今天': 1,
         '今年': 1,
         '今晚': 1,
         '共': 1,
         '分鐘': 2,
         '到來': 1,
         '即日': 1,
         '台北': 3,
         '報時': 1,
         '外': 1,
         '天': 1,
         '天際線': 1,
         '將': 1,
         '延續': 1,
         '從': 1,
         '持續': 1,
         '推出': 1,
         '新春': 2,
         '新聞稿': 1,
         '日': 2,
         '時': 2,
         '會': 1,
         '月': 2,
         '有': 2,
         '每': 1,
         '每晚': 1,
         '燈光': 1,
         '燈光秀': 3,
         '特別': 1,
         '的': 1,
         '至': 3,
         '虎年': 3,
         '表示': 2,
         '設計': 1,
         '變化': 1,
         '起': 1,
         '迎接': 1,
         '透過': 1,
         '還': 1,
         '閃耀': 1,
         '開始': 1,
         '除整': 1,
         '點會': 1})

In [43]:
counter.most_common(5)

[('虎年', 3), ('台北', 3), ('燈光秀', 3), ('至', 3), ('表示', 2)]

In [44]:
counter_series = pd.Series(counter).sort_values(ascending=False)
fig = counter_series.plot.bar()
fig

# 詞彙分級

處理`TOCFL華語詞彙分級表`的方法詳見[這裡](https://colab.research.google.com/github/howard-haowen/NLP-demos/blob/main/TOCFL_wordlist.ipynb)。

# 互動介面

In [57]:
from ipywidgets import interact

In [69]:
def show_tokens(input_text):
    doc = nlp(input_text)
    for token in doc:
        print(token.text, end=" | ")

In [70]:
def show_ner(input_text):
    doc = nlp(input_text)
    displacy.render(doc, style='ent',jupyter=True)

In [75]:
def filter_tokens(doc):
    clean_tokens = [tok for tok in doc if tok.pos_ not in PUNCT_SYM]
    clean_tokens = (
        [tok for tok in clean_tokens if 
         not tok.like_email and 
         not tok.like_num and 
         not tok.like_url and 
         not tok.is_space]
    )
    return clean_tokens

In [78]:
def show_aided_text(input_text, pronunciation="pinyin"):
    PUNCT_SYM = ["PUNCT", "SYM"]
    TOK_SEP = " | "
    doc = nlp(input_text)
    for idx, sent in enumerate(doc.sents):
        tokens_text = [tok.text for tok in sent if tok.pos_ not in PUNCT_SYM]
        pinyins = [hanzi.to_pinyin(word) for word in tokens_text]
        sounds = pinyins
        if pronunciation == "bpm":
            zhuyins = [transcriptions.pinyin_to_zhuyin(word) for word in pinyins]
            sounds = zhuyins
        elif pronunciation == "ipa":
            ipas = [transcriptions.pinyin_to_ipa(word) for word in pinyins]
            sounds = ipas

        display = []
        for text, sound in zip(tokens_text, sounds):
            res = f"{text} [{sound}]"
            display.append(res)
        if display:
            display_text = TOK_SEP.join(display)
            print(f"{idx+1} >>> {display_text}")
        else:
            print(f"{idx+1} >>> EMPTY LINE")

In [73]:
text = "阿伯說他要從台北騎腳踏車到高雄。"
interact(show_tokens, input_text=text)

interactive(children=(Text(value='阿伯說他要從台北騎腳踏車到高雄。', description='input_text'), Output()), _dom_classes=('widg…

<function __main__.show_tokens>

In [74]:
interact(show_ner, input_text=text)

interactive(children=(Text(value='阿伯說他要從台北騎腳踏車到高雄。', description='input_text'), Output()), _dom_classes=('widg…

<function __main__.show_ner>

In [80]:
interact(show_aided_text, 
           input_text=text, 
           pronunciation='pinyin')

interactive(children=(Text(value='阿伯說他要從台北騎腳踏車到高雄。', description='input_text'), Text(value='pinyin', descripti…

<function __main__.show_aided_text>

In [ ]:
interact(moedict_caller, word="高興")